In [2]:
from google.colab import files
uploaded = files.upload()

Saving CSIData.zip to CSIData.zip


In [3]:
import zipfile
import pandas as pd
from io import BytesIO
import numpy as np
import re

#zip = 'C://Users//Lance//Documents//UNO//ECON8310 Economic Forecasting//CSIData//CSIData.zip' --- I added the zip folder to colab so I could run you code
zip = 'CSIData.zip'
all_dfs = []

with zipfile.ZipFile(zip, 'r') as z:
    for file in z.namelist():
        if '__MACOSX' not in file and file.endswith(('.xlsx', '.xls')):
            with z.open(file) as f:
                df = pd.read_excel(f, skiprows=5, engine='openpyxl')
                df = df.drop(0)
                df = df.drop(['First Name','Last Name','External Student ID','Tags'], axis=1)
                df['source_file'] = file

                #this section below needs to be done here (before the append) to avoid a mismatch of columns
                columns = df.columns.tolist()
                class_identifiers = ['Record ID','Student Status','Room','source_file']

                for i in range(1, len(columns)):
                    if "Unnamed" in columns[i]:
                        columns[i] = str(columns[i - 1]) + ' OUT'

                for i in range(1, len(columns)):
                    if "OUT" not in columns[i] and columns[i] not in class_identifiers:
                        columns[i] = str(columns[i]) + ' IN'
                #end of section

                columns = [col.replace('Jan', '01') for col in columns]
                columns = [col.replace('Feb', '02') for col in columns]
                columns = [col.replace('Mar', '03') for col in columns]
                columns = [col.replace('Apr', '04') for col in columns]
                columns = [col.replace('May', '05') for col in columns]
                columns = [col.replace('Jun', '06') for col in columns]
                columns = [col.replace('Jul', '07') for col in columns]
                columns = [col.replace('Aug', '08') for col in columns]
                columns = [col.replace('Sep', '09') for col in columns]
                columns = [col.replace('Oct', '10') for col in columns]
                columns = [col.replace('Nov', '11') for col in columns]
                columns = [col.replace('Dec', '12') for col in columns]
                columns = [col.replace(' ', '-')    for col in columns]

                columns[0] = 'Record ID'
                columns[1] = 'Student Status'

                df.columns = columns

                all_dfs.append(df)
combined_df = pd.concat(all_dfs, ignore_index=True)
combined_df_backup = combined_df


In [4]:
#regex looking for the times in the date columns
#time_pattern = r'(\d{1,2}:\d{2} [APM]{2})'
time_pattern = r'(\b(?:1[0-2]|0?[1-9]):[0-5][0-9]\s?(?:AM|PM)\b)'

#removing everything but the times in the date columns. Converts blank values to null
for col in combined_df.columns:
    if col not in class_identifiers:
        combined_df[col] = combined_df[col].apply(lambda x:re.search(time_pattern,str(x)).group(1) if isinstance(x,str) and re.search(time_pattern,str(x)) else np.nan)
        combined_df[col] = combined_df[col].apply(lambda x:pd.to_datetime(str(x), format='%I:%M %p') if isinstance(x,str) else np.nan)

In [5]:
#Validating the results
# combined_df[(combined_df['Record ID'] == 'KL100212') & (combined_df['source_file'] == 'ECEC 2024 Student Sign In and Out.xlsx')][['Record ID','Room','03-04-IN', '03-04-OUT','source_file']]

In [6]:
#Adding a unique identifier for each row in the file that is unique to Record ID, Student Status, Room, and Tags. Without this, we cannot use the pd.melt function because it finds duplicate values for students that check in/out more than 1 time in a day.
combined_df['record_nbr'] = combined_df.groupby(['Record ID','Student Status','Room','source_file']).cumcount() + 1

#split the columns into two groups so the date columns can be transposed into rows
id_vars = ['Record ID','Student Status','Room','record_nbr','source_file']
time_columns = [col for col in combined_df.columns if col not in id_vars]

df_melted = combined_df.melt(id_vars=id_vars, value_vars=time_columns, var_name="Date_Time", value_name="Time")

#remove records where there is no data
df_melted = df_melted[df_melted['Time'].notna()]
df_melted['Time'] = pd.to_datetime(df_melted['Time'], format='%H:%M').dt.time
df_melted

<ipython-input-6-e33e87a4a3dc>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_df['record_nbr'] = combined_df.groupby(['Record ID','Student Status','Room','source_file']).cumcount() + 1


,Record ID,Student Status,Room,record_nbr,source_file,Date_Time,Time
6420,KL100303,Active,Henry Infants,1,ECEC 2024 Student Sign In and Out.xlsx,01-02-IN,08:11:00
6426,KL100305,Active,Pennie Toddlers,1,ECEC 2024 Student Sign In and Out.xlsx,01-02-IN,08:46:00
6429,KL100233,Active,Henry Toddlers,1,ECEC 2024 Student Sign In and Out.xlsx,01-02-IN,08:44:00
6432,KL100281,Active,Pennie Preschool 1,1,ECEC 2024 Student Sign In and Out.xlsx,01-02-IN,07:39:00
6438,KL100267,Active,Grampy Tom Toddlers,1,ECEC 2024 Student Sign In and Out.xlsx,01-02-IN,09:23:00
...,...,...,...,...,...,...,...
2111526,KL100200,Graduate,Dinosaur Stomp,1,Spellman CDC 2024 Student Sign in and out.xlsx,02-29-OUT,16:29:00
2111527,KL100198,Inactive,Wild Things,1,Spellman CDC 2024 Student Sign in and out.xlsx,02-29-OUT,17:52:00
2111532,KL100363,Inactive,Goodnight Moon,1,Spellman CDC 2024 Student Sign in and out.xlsx,02-29-OUT,16:15:00
2111539,KL100284,Inactive,Rainbow Fish,1,Spellman CDC 2024 Student Sign in and out.xlsx,02-29-OUT,16:38:00


In [7]:
df_melted[['Date', 'In_Out']] = df_melted['Date_Time'].str.extract(r'(\d{2}-\d{2})-(IN|OUT)')
df_melted

,Record ID,Student Status,Room,record_nbr,source_file,Date_Time,Time,Date,In_Out
6420,KL100303,Active,Henry Infants,1,ECEC 2024 Student Sign In and Out.xlsx,01-02-IN,08:11:00,01-02,IN
6426,KL100305,Active,Pennie Toddlers,1,ECEC 2024 Student Sign In and Out.xlsx,01-02-IN,08:46:00,01-02,IN
6429,KL100233,Active,Henry Toddlers,1,ECEC 2024 Student Sign In and Out.xlsx,01-02-IN,08:44:00,01-02,IN
6432,KL100281,Active,Pennie Preschool 1,1,ECEC 2024 Student Sign In and Out.xlsx,01-02-IN,07:39:00,01-02,IN
6438,KL100267,Active,Grampy Tom Toddlers,1,ECEC 2024 Student Sign In and Out.xlsx,01-02-IN,09:23:00,01-02,IN
...,...,...,...,...,...,...,...,...,...
2111526,KL100200,Graduate,Dinosaur Stomp,1,Spellman CDC 2024 Student Sign in and out.xlsx,02-29-OUT,16:29:00,02-29,OUT
2111527,KL100198,Inactive,Wild Things,1,Spellman CDC 2024 Student Sign in and out.xlsx,02-29-OUT,17:52:00,02-29,OUT
2111532,KL100363,Inactive,Goodnight Moon,1,Spellman CDC 2024 Student Sign in and out.xlsx,02-29-OUT,16:15:00,02-29,OUT
2111539,KL100284,Inactive,Rainbow Fish,1,Spellman CDC 2024 Student Sign in and out.xlsx,02-29-OUT,16:38:00,02-29,OUT


In [8]:
df_final = df_melted.pivot(index=['Record ID','Student Status','Room', 'Date','record_nbr','source_file'], columns='In_Out', values='Time').reset_index()
df_final.rename(columns={'IN': 'Time In', 'OUT': 'Time Out'}, inplace=True)
df_final

In_Out,Record ID,Student Status,Room,Date,record_nbr,source_file,Time In,Time Out
0,KL100004,Graduate,Rainbow Fish,01-03,1,Spellman CDC 2022 Student Sign in and out.xlsx,07:56:00,17:29:00
1,KL100004,Graduate,Rainbow Fish,01-03,1,Spellman CDC 2023 Student Sign in and out.xlsx,07:33:00,17:16:00
2,KL100004,Graduate,Rainbow Fish,01-04,1,Spellman CDC 2022 Student Sign in and out.xlsx,07:29:00,17:24:00
3,KL100004,Graduate,Rainbow Fish,01-04,1,Spellman CDC 2023 Student Sign in and out.xlsx,07:46:00,17:23:00
4,KL100004,Graduate,Rainbow Fish,01-05,1,Spellman CDC 2022 Student Sign in and out.xlsx,07:34:00,17:32:00
...,...,...,...,...,...,...,...,...
131906,KL100470,Active,Goodnight Moon,02-24,1,Spellman CDC 2025 01012025-02282025 Student Si...,08:05:00,14:52:00
131907,KL100470,Active,Goodnight Moon,02-25,1,Spellman CDC 2025 01012025-02282025 Student Si...,07:51:00,15:22:00
131908,KL100470,Active,Goodnight Moon,02-26,1,Spellman CDC 2025 01012025-02282025 Student Si...,08:46:00,14:55:00
131909,KL100470,Active,Goodnight Moon,02-27,1,Spellman CDC 2025 01012025-02282025 Student Si...,07:32:00,15:50:00


In [9]:
missing_time_out = df_final['Time Out'].isna().sum()
print("Missing Time Out:", missing_time_out)

missing_time_in = df_final['Time In'].isna().sum()
print("Missing Time In:", missing_time_in)

Missing Time Out: 993
Missing Time In: 0


In [10]:
#may want to add a step here to take an avg. Time out value by Record ID and populate null values (although that may not help with counts?)

In [11]:
#Should probably exclude these? or set the Time Out to 5:00 PM ? Probably depends how many records in total as % of total population
df_final[df_final['Time Out'].isna()]

df_final = df_final.dropna(subset=['Time Out'])
df_final = df_final.dropna(subset=['Time In'])
df_final

In_Out,Record ID,Student Status,Room,Date,record_nbr,source_file,Time In,Time Out
0,KL100004,Graduate,Rainbow Fish,01-03,1,Spellman CDC 2022 Student Sign in and out.xlsx,07:56:00,17:29:00
1,KL100004,Graduate,Rainbow Fish,01-03,1,Spellman CDC 2023 Student Sign in and out.xlsx,07:33:00,17:16:00
2,KL100004,Graduate,Rainbow Fish,01-04,1,Spellman CDC 2022 Student Sign in and out.xlsx,07:29:00,17:24:00
3,KL100004,Graduate,Rainbow Fish,01-04,1,Spellman CDC 2023 Student Sign in and out.xlsx,07:46:00,17:23:00
4,KL100004,Graduate,Rainbow Fish,01-05,1,Spellman CDC 2022 Student Sign in and out.xlsx,07:34:00,17:32:00
...,...,...,...,...,...,...,...,...
131906,KL100470,Active,Goodnight Moon,02-24,1,Spellman CDC 2025 01012025-02282025 Student Si...,08:05:00,14:52:00
131907,KL100470,Active,Goodnight Moon,02-25,1,Spellman CDC 2025 01012025-02282025 Student Si...,07:51:00,15:22:00
131908,KL100470,Active,Goodnight Moon,02-26,1,Spellman CDC 2025 01012025-02282025 Student Si...,08:46:00,14:55:00
131909,KL100470,Active,Goodnight Moon,02-27,1,Spellman CDC 2025 01012025-02282025 Student Si...,07:32:00,15:50:00


In [12]:
unique_rooms = df_final['Room'].unique().tolist()

# Mapping of room names to age levels
room_to_level = {
    'Goodnight Moon': 'Infant',
    'Pandas': 'Toddler',
    'House Pooh': 'Infant',
    'Llamas Llamas': 'Multi-Age',
    'Hungry Caterpillars': 'Multi-Age',
    'Rabbits': 'Toddler',
    'Wild Things': 'Preschool',
    'Monkeys': 'Toddler',
    'Rainbow Fish': 'Preschool',
    'Dinosaurs': 'Pre-K',

    'Camp':'???', #'Pre-K'
    'Dinosaur Stomp':'Pre-K',
    'Grampy Tom Multi-Age':'???', #'Pre-K'
    'Grampy Tom Preschool': 'Preschool',
    'Grampy Tom Toddlers': 'Preschool',
    'Henry Infants': 'Infant',
    'Henry Multi-Age':'???', #'Pre-K'
    'Henry Toddlers': 'Toddler',
    'Pennie Infants': 'Infant',
    'Pennie Preschool 1': 'Preschool',
    'Pennie Toddlers': 'Toddler',
    'Pre-K1': 'Pre-K',
    'Pre-K2': 'Pre-K'
    }

# Mapping of age levels to ratios
level_to_ratio = {
    'Infant': '4', #4:1
    'Multi-Age': '4', #4:1
    'Toddler': '6', #6:1
    'Preschool': '10', #10:1
    'Pre-K': '12' #12:1
}

# Add level column
df_final['Level'] = df_final['Room'].map(room_to_level)

# Add ratio column
df_final['Students per Teacher'] = df_final['Level'].map(level_to_ratio)
df_final

In_Out,Record ID,Student Status,Room,Date,record_nbr,source_file,Time In,Time Out,Level,Students per Teacher
0,KL100004,Graduate,Rainbow Fish,01-03,1,Spellman CDC 2022 Student Sign in and out.xlsx,07:56:00,17:29:00,Preschool,10
1,KL100004,Graduate,Rainbow Fish,01-03,1,Spellman CDC 2023 Student Sign in and out.xlsx,07:33:00,17:16:00,Preschool,10
2,KL100004,Graduate,Rainbow Fish,01-04,1,Spellman CDC 2022 Student Sign in and out.xlsx,07:29:00,17:24:00,Preschool,10
3,KL100004,Graduate,Rainbow Fish,01-04,1,Spellman CDC 2023 Student Sign in and out.xlsx,07:46:00,17:23:00,Preschool,10
4,KL100004,Graduate,Rainbow Fish,01-05,1,Spellman CDC 2022 Student Sign in and out.xlsx,07:34:00,17:32:00,Preschool,10
...,...,...,...,...,...,...,...,...,...,...
131906,KL100470,Active,Goodnight Moon,02-24,1,Spellman CDC 2025 01012025-02282025 Student Si...,08:05:00,14:52:00,Infant,4
131907,KL100470,Active,Goodnight Moon,02-25,1,Spellman CDC 2025 01012025-02282025 Student Si...,07:51:00,15:22:00,Infant,4
131908,KL100470,Active,Goodnight Moon,02-26,1,Spellman CDC 2025 01012025-02282025 Student Si...,08:46:00,14:55:00,Infant,4
131909,KL100470,Active,Goodnight Moon,02-27,1,Spellman CDC 2025 01012025-02282025 Student Si...,07:32:00,15:50:00,Infant,4


In [13]:
df_final = df_final[~df_final['Room'].isin(['Camp', 'Grampy Tom Multi-Age','Henry Multi-Age'])]

In [14]:
#Create new column so we can convert to time, and then remove 30 minutes
df_final['Time In (Less 30 minutes)'] = pd.to_datetime(df_final['Time In'],format='%H:%M:%S')
#remove 30 minutes from time
df_final['Time In (Less 30 minutes)'] = df_final['Time In (Less 30 minutes)'] + pd.Timedelta(minutes=-30)
#Convert back to time format
df_final['Time In (Less 30 minutes)'] = pd.to_datetime(df_final['Time In (Less 30 minutes)'], format='%H:%M').dt.time
df_final


<ipython-input-14-73cb261667c4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Time In (Less 30 minutes)'] = pd.to_datetime(df_final['Time In'],format='%H:%M:%S')
<ipython-input-14-73cb261667c4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Time In (Less 30 minutes)'] = df_final['Time In (Less 30 minutes)'] + pd.Timedelta(minutes=-30)
<ipython-input-14-73cb261667c4>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In_Out,Record ID,Student Status,Room,Date,record_nbr,source_file,Time In,Time Out,Level,Students per Teacher,Time In (Less 30 minutes)
0,KL100004,Graduate,Rainbow Fish,01-03,1,Spellman CDC 2022 Student Sign in and out.xlsx,07:56:00,17:29:00,Preschool,10,07:26:00
1,KL100004,Graduate,Rainbow Fish,01-03,1,Spellman CDC 2023 Student Sign in and out.xlsx,07:33:00,17:16:00,Preschool,10,07:03:00
2,KL100004,Graduate,Rainbow Fish,01-04,1,Spellman CDC 2022 Student Sign in and out.xlsx,07:29:00,17:24:00,Preschool,10,06:59:00
3,KL100004,Graduate,Rainbow Fish,01-04,1,Spellman CDC 2023 Student Sign in and out.xlsx,07:46:00,17:23:00,Preschool,10,07:16:00
4,KL100004,Graduate,Rainbow Fish,01-05,1,Spellman CDC 2022 Student Sign in and out.xlsx,07:34:00,17:32:00,Preschool,10,07:04:00
...,...,...,...,...,...,...,...,...,...,...,...
131906,KL100470,Active,Goodnight Moon,02-24,1,Spellman CDC 2025 01012025-02282025 Student Si...,08:05:00,14:52:00,Infant,4,07:35:00
131907,KL100470,Active,Goodnight Moon,02-25,1,Spellman CDC 2025 01012025-02282025 Student Si...,07:51:00,15:22:00,Infant,4,07:21:00
131908,KL100470,Active,Goodnight Moon,02-26,1,Spellman CDC 2025 01012025-02282025 Student Si...,08:46:00,14:55:00,Infant,4,08:16:00
131909,KL100470,Active,Goodnight Moon,02-27,1,Spellman CDC 2025 01012025-02282025 Student Si...,07:32:00,15:50:00,Infant,4,07:02:00


In [15]:
df_final['Location'] = df_final['source_file'].apply(lambda x: 'Spellman' if 'spellman' in str(x).lower() else 'ECEC' if 'ecec' in str(x).lower() else 'Other')
df_final['Year'] = df_final['source_file'].str.extract(r'(20\d{2})')
df_final['Full Date'] = pd.to_datetime(df_final['Date'].astype(str) + '-' + df_final['Year'].astype(str), format='%m-%d-%Y')
df_final


<ipython-input-15-e16ccfb2f6fb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Location'] = df_final['source_file'].apply(lambda x: 'Spellman' if 'spellman' in str(x).lower() else 'ECEC' if 'ecec' in str(x).lower() else 'Other')
<ipython-input-15-e16ccfb2f6fb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Year'] = df_final['source_file'].str.extract(r'(20\d{2})')
<ipython-input-15-e16ccfb2f6fb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In_Out,Record ID,Student Status,Room,Date,record_nbr,source_file,Time In,Time Out,Level,Students per Teacher,Time In (Less 30 minutes),Location,Year,Full Date
0,KL100004,Graduate,Rainbow Fish,01-03,1,Spellman CDC 2022 Student Sign in and out.xlsx,07:56:00,17:29:00,Preschool,10,07:26:00,Spellman,2022,2022-01-03
1,KL100004,Graduate,Rainbow Fish,01-03,1,Spellman CDC 2023 Student Sign in and out.xlsx,07:33:00,17:16:00,Preschool,10,07:03:00,Spellman,2023,2023-01-03
2,KL100004,Graduate,Rainbow Fish,01-04,1,Spellman CDC 2022 Student Sign in and out.xlsx,07:29:00,17:24:00,Preschool,10,06:59:00,Spellman,2022,2022-01-04
3,KL100004,Graduate,Rainbow Fish,01-04,1,Spellman CDC 2023 Student Sign in and out.xlsx,07:46:00,17:23:00,Preschool,10,07:16:00,Spellman,2023,2023-01-04
4,KL100004,Graduate,Rainbow Fish,01-05,1,Spellman CDC 2022 Student Sign in and out.xlsx,07:34:00,17:32:00,Preschool,10,07:04:00,Spellman,2022,2022-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131906,KL100470,Active,Goodnight Moon,02-24,1,Spellman CDC 2025 01012025-02282025 Student Si...,08:05:00,14:52:00,Infant,4,07:35:00,Spellman,2025,2025-02-24
131907,KL100470,Active,Goodnight Moon,02-25,1,Spellman CDC 2025 01012025-02282025 Student Si...,07:51:00,15:22:00,Infant,4,07:21:00,Spellman,2025,2025-02-25
131908,KL100470,Active,Goodnight Moon,02-26,1,Spellman CDC 2025 01012025-02282025 Student Si...,08:46:00,14:55:00,Infant,4,08:16:00,Spellman,2025,2025-02-26
131909,KL100470,Active,Goodnight Moon,02-27,1,Spellman CDC 2025 01012025-02282025 Student Si...,07:32:00,15:50:00,Infant,4,07:02:00,Spellman,2025,2025-02-27


In [16]:
#final_counts_rev = final_counts[final_counts['Student Count'] > 0]
#final_counts_rev[final_counts_rev['Time Bucket'] == '19:00'].shape[0] #15 instances where there are kids still checked in after 7:00 PM. For predictive analysis, we can consider these outliers, and do not need to forecast staffing past 7:00 PM

In [17]:
# Generate 30-minute time buckets (from 06:00 to 19:00)
time_buckets = pd.date_range("06:00", "19:00", freq="30min").time
bucket_labels = [t.strftime('%H:%M') for t in time_buckets]

# Function to check which time bucket a student's time_in and time_out falls into
def count_students_in_bucket(row):
    time_in = row['Time In (Less 30 minutes)']
    time_out = row['Time Out']

    # Create a boolean mask to check the student's presence in each time bucket
    presence = []
    for t in time_buckets:
        if time_in <= t < time_out:
            presence.append(True)
        else:
            presence.append(False)
    return presence

# Apply count_students_in_bucket function and expand the result into individual time buckets
df_final['Student Present in Buckets'] = df_final.apply(count_students_in_bucket, axis=1)

# Add the time bucket labels to the DataFrame based on presence
df_final['Time Bucket'] = df_final['Student Present in Buckets'].apply(
    lambda presence: [bucket_labels[i] for i, p in enumerate(presence) if p]
)

# Now explode the 'Time Bucket' column to create separate rows for each time bucket
df_final_exploded = df_final.explode('Time Bucket')

# Drop duplicates to avoid counting the same student multiple times for the same bucket
df_final_exploded = df_final_exploded.drop_duplicates(subset=['Record ID', 'Room', 'Full Date', 'Time Bucket'])

# Group by Room, Date, and Time Bucket, then count the students
final_counts = df_final_exploded.groupby(['Room', 'Full Date', 'Time Bucket','Level','Students per Teacher','Location']).size().reset_index(name='Student Count')

final_counts


<ipython-input-17-d0ad5c28e9c1>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Student Present in Buckets'] = df_final.apply(count_students_in_bucket, axis=1)
<ipython-input-17-d0ad5c28e9c1>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Time Bucket'] = df_final['Student Present in Buckets'].apply(


,Room,Full Date,Time Bucket,Level,Students per Teacher,Location,Student Count
0,Dinosaur Stomp,2022-01-03,07:00,Pre-K,12,Spellman,1
1,Dinosaur Stomp,2022-01-03,07:30,Pre-K,12,Spellman,5
2,Dinosaur Stomp,2022-01-03,08:00,Pre-K,12,Spellman,14
3,Dinosaur Stomp,2022-01-03,08:30,Pre-K,12,Spellman,20
4,Dinosaur Stomp,2022-01-03,09:00,Pre-K,12,Spellman,21
...,...,...,...,...,...,...,...
264676,Wild Things,2025-02-28,15:00,Preschool,10,Spellman,18
264677,Wild Things,2025-02-28,15:30,Preschool,10,Spellman,14
264678,Wild Things,2025-02-28,16:00,Preschool,10,Spellman,13
264679,Wild Things,2025-02-28,16:30,Preschool,10,Spellman,12


In [18]:
#validating data
#final_counts[(final_counts['Room']=='Wild Things') & (final_counts['Full Date']=='2025-02-28')]
final_counts[(final_counts['Room']=='Pennie Preschool 1') & (final_counts['Full Date']=='2024-03-04')]

,Room,Full Date,Time Bucket,Level,Students per Teacher,Location,Student Count
164905,Pennie Preschool 1,2024-03-04,06:30,Preschool,10,ECEC,1
164906,Pennie Preschool 1,2024-03-04,07:00,Preschool,10,ECEC,1
164907,Pennie Preschool 1,2024-03-04,07:30,Preschool,10,ECEC,5
164908,Pennie Preschool 1,2024-03-04,08:00,Preschool,10,ECEC,8
164909,Pennie Preschool 1,2024-03-04,08:30,Preschool,10,ECEC,13
164910,Pennie Preschool 1,2024-03-04,09:00,Preschool,10,ECEC,13
164911,Pennie Preschool 1,2024-03-04,09:30,Preschool,10,ECEC,13
164912,Pennie Preschool 1,2024-03-04,10:00,Preschool,10,ECEC,13
164913,Pennie Preschool 1,2024-03-04,10:30,Preschool,10,ECEC,13
164914,Pennie Preschool 1,2024-03-04,11:00,Preschool,10,ECEC,13


In [19]:
# 1) make sure Full Date is a date (no time component)
final_counts['Full Date'] = pd.to_datetime(final_counts['Full Date']).dt.date

# 2) combine date + time into a single datetime column 'ds'
final_counts['ds'] = pd.to_datetime(
    final_counts['Full Date'].astype(str) + ' ' +
    final_counts['Time Bucket']          # adjust name if yours is different
)

# 3) optional: verify
print(final_counts.head())

             Room   Full Date Time Bucket  Level Students per Teacher  \
0  Dinosaur Stomp  2022-01-03       07:00  Pre-K                   12   
1  Dinosaur Stomp  2022-01-03       07:30  Pre-K                   12   
2  Dinosaur Stomp  2022-01-03       08:00  Pre-K                   12   
3  Dinosaur Stomp  2022-01-03       08:30  Pre-K                   12   
4  Dinosaur Stomp  2022-01-03       09:00  Pre-K                   12   

   Location  Student Count                  ds  
0  Spellman              1 2022-01-03 07:00:00  
1  Spellman              5 2022-01-03 07:30:00  
2  Spellman             14 2022-01-03 08:00:00  
3  Spellman             20 2022-01-03 08:30:00  
4  Spellman             21 2022-01-03 09:00:00  


In [20]:
#TYPICAL WEEK FORECAST

import pandas as pd
from prophet import Prophet
import numpy as np

# Student to Teacher Ratio Matrix
singular_map = {
    'Infant':    4,
    'Multi-Age': 4,
    'Toddler':   6,
    'Preschool':10,
    'Pre-K':    12
}

# find the next Monday
last_date    = final_counts['ds'].max()
days_to_mon  = (7 - last_date.weekday()) % 7 or 7
start_monday = (last_date + pd.Timedelta(days=days_to_mon)).normalize()

# full 7-day, half-hour index
full_slots  = pd.date_range(start_monday, periods=7*48, freq='30min')
future_full = pd.DataFrame({'ds': full_slots})

typical_week_preds = {}
staff_typical      = {}

for room in final_counts['Room'].unique():
    #  prepare & zero-fill training series
    df_r = (
        final_counts
        .loc[final_counts['Room']==room, ['ds','Student Count']]
        .rename(columns={'Student Count':'y'})
    )
    full_idx = pd.date_range(df_r['ds'].min(), df_r['ds'].max(), freq='30min')
    df_r = (
        df_r.set_index('ds')
            .reindex(full_idx, fill_value=0)
            .reset_index()
            .rename(columns={'index':'ds'})
    )

    # fit prophet with custom seasonality - Was not able to get it looking good with standard seasonality
    m = Prophet(
        daily_seasonality=False,
        weekly_seasonality=False,
        yearly_seasonality=False
    )
    m.add_seasonality(name='daily',  period=1, fourier_order=20)
    m.add_seasonality(name='weekly', period=7, fourier_order= 8)

    m.fit(df_r)

    # seasonal‐only forecast on full week
    fc = m.predict(future_full)
    fc['seasonal'] = fc['daily'] + fc['weekly']
    typ = fc[['ds','seasonal']].rename(columns={'seasonal':'yhat'})

    # floor negatives, then zero out nights & weekends
    typ['yhat'] = typ['yhat'].clip(lower=0)
    mask_night = (typ['ds'].dt.hour < 6) | (typ['ds'].dt.hour >= 18)
    mask_weekend = typ['ds'].dt.dayofweek >= 5
    typ.loc[mask_night | mask_weekend, 'yhat'] = 0

    typical_week_preds[room] = typ

    # staffing need
    lvl   = final_counts.loc[final_counts['Room']==room, 'Level'].iloc[0]
    ratio = singular_map[lvl.rstrip('s')]
    staff_typical[room] = (
        typ
        .assign(staff_needed=lambda d: np.ceil(d['yhat']/ratio).astype(int))
        [['ds','staff_needed']]
    )



DEBUG:cmdstanpy:input tempfile: /tmp/tmpsllfw42c/bfdtmlcm.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpsllfw42c/7ys6v_u8.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=31974', 'data', 'file=/tmp/tmpsllfw42c/bfdtmlcm.json', 'init=/tmp/tmpsllfw42c/7ys6v_u8.json', 'output', 'file=/tmp/tmpsllfw42c/prophet_modelm4zj5cah/prophet_model-20250504143039.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:30:39 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:31:39 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpsllfw42c/ny2l_6e9.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpsllfw42c/_pejo_9m.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

In [21]:
#NEXT WEEK FORECAST

# Student to teacher ratio matrix
singular_map = {
    'Infant':    4,
    'Multi-Age': 4,
    'Toddler':   6,
    'Preschool':10,
    'Pre-K':    12
}

# Finds the first monday after the historical data
last_date    = final_counts['ds'].max()
days_to_mon  = (7 - last_date.weekday()) % 7 or 7
start_monday = (last_date + pd.Timedelta(days=days_to_mon)).normalize()

full_slots  = pd.date_range(start_monday, periods=7*48, freq='30min')
future_full = pd.DataFrame({'ds': full_slots})

next_week_preds = {}
staff_next      = {}

for room in final_counts['Room'].unique():
    # prep & zero-fill training
    df_r = (
        final_counts
        .loc[final_counts['Room']==room, ['ds','Student Count']]
        .rename(columns={'Student Count':'y'})
    )
    full_idx = pd.date_range(df_r['ds'].min(), df_r['ds'].max(), freq='30min')
    df_r = (
        df_r.set_index('ds')
            .reindex(full_idx, fill_value=0)
            .reset_index()
            .rename(columns={'index':'ds'})
    )

    # Same as above - still looking at how to do this best
    m = Prophet(
        daily_seasonality=False,
        weekly_seasonality=False,
        yearly_seasonality=False
    )
    m.add_seasonality('daily',  period=1, fourier_order=20)
    m.add_seasonality('weekly', period=7, fourier_order= 8)
    m.add_seasonality('monthly',period=30.5, fourier_order=5)
    m.fit(df_r)

    # forecast both trend and seasonal
    fc  = m.predict(future_full)
    nxt = fc[['ds','yhat']].copy()

    # floor negatives, zero nights/weekends
    nxt['yhat'] = nxt['yhat'].clip(lower=0)
    mask_night = (nxt['ds'].dt.hour < 6) | (nxt['ds'].dt.hour >= 18)
    mask_weekend = nxt['ds'].dt.dayofweek >= 5
    nxt.loc[mask_night | mask_weekend, 'yhat'] = 0

    next_week_preds[room] = nxt

    # staffing need
    lvl   = final_counts.loc[final_counts['Room']==room, 'Level'].iloc[0]
    ratio = singular_map[lvl.rstrip('s')]
    staff_next[room] = (
        nxt
        .assign(staff_needed=lambda d: np.ceil(d['yhat']/ratio).astype(int))
        [['ds','staff_needed']]
    )



DEBUG:cmdstanpy:input tempfile: /tmp/tmpsllfw42c/v6up82u5.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpsllfw42c/8j9e3vys.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=83455', 'data', 'file=/tmp/tmpsllfw42c/v6up82u5.json', 'init=/tmp/tmpsllfw42c/8j9e3vys.json', 'output', 'file=/tmp/tmpsllfw42c/prophet_modeld_ra74_6/prophet_model-20250504144631.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:46:31 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:47:39 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpsllfw42c/v30s4hb5.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpsllfw42c/zdeiumbt.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

In [30]:
from google.colab import drive
drive.mount('/content/drive')

# 2) Make an output directory
import os
out_dir = '/content/drive/MyDrive/CSI_forecasts'
os.makedirs(out_dir, exist_ok=True)

# 3) Save all forecasts
def clean_name(s):
    return s.replace(' ', '_').replace('/', '_')

# Typical week attendance & staffing
for room, df in typical_week_preds.items():
    base = clean_name(room)
    df.to_csv(f"{out_dir}/{base}_typical_attendance.csv", index=False)
    staff_typical[room].to_csv(f"{out_dir}/{base}_typical_staff.csv", index=False)

# Next week attendance & staffing
for room, df in next_week_preds.items():
    base = clean_name(room)
    df.to_csv(f"{out_dir}/{base}_next_attendance.csv", index=False)
    staff_next[room].to_csv(f"{out_dir}/{base}_next_staff.csv", index=False)

print("Saved all forecasts to", out_dir)


Mounted at /content/drive
Saved all forecasts to /content/drive/MyDrive/CSI_forecasts


In [22]:
#Single Room Typical Week Attendance

import plotly.express as px


# Used this to switch through rooms for a sanity check
room = list(typical_week_preds.keys())[0]
df_typ = typical_week_preds[room]

fig = px.line(
    df_typ,
    x='ds',
    y='yhat',
    title=f"Typical Week Forecast — {room}",
    labels={'ds':'Datetime', 'yhat':'Predicted Attendance'}
)
fig.update_xaxes(tickangle=45)
fig.show()

In [23]:
# Next week single room attendance

room   = list(next_week_preds.keys())[0]
df_nxt = next_week_preds[room]

fig = px.line(
    df_nxt,
    x='ds',
    y='yhat',
    title=f"Next Week Forecast — {room}",
    labels={'ds':'Datetime', 'yhat':'Predicted Attendance'}
)
fig.update_xaxes(tickangle=45)
fig.show()

In [31]:
#Saving forecast results to repository to eliminate the need to rerun forecast

import os

out_dir = 'CSI_forecasts'
os.makedirs(out_dir, exist_ok=True)

def clean_name(s):
    return s.replace(' ', '_').replace('/', '_')

# 2) save typical‐week results
for room, df in typical_week_preds.items():
    base = clean_name(room)
    df.to_csv(os.path.join(out_dir, f"{base}_typical_attendance.csv"), index=False)
    staff_typical[room].to_csv(
        os.path.join(out_dir, f"{base}_typical_staff.csv"), index=False
    )

# 3) save next‐week results
for room, df in next_week_preds.items():
    base = clean_name(room)
    df.to_csv(os.path.join(out_dir, f"{base}_next_attendance.csv"), index=False)
    staff_next[room].to_csv(
        os.path.join(out_dir, f"{base}_next_staff.csv"), index=False
    )

print(f"Forecast CSVs written to ./{out_dir}/")


Forecast CSVs written to ./CSI_forecasts/


In [25]:
#Plot to see total staffing needs for both typical and next week
import pandas as pd

# 1) Stack all rooms into one DataFrame with a 'Room' column
df_all_typ = pd.concat(
    [v.assign(Room=k) for k,v in staff_typical.items()],
    ignore_index=True
)
df_all_nxt = pd.concat(
    [v.assign(Room=k) for k,v in staff_next.items()],
    ignore_index=True
)

# 2) Sum across rooms for each timeslot
total_typ = (
    df_all_typ
    .groupby('ds', as_index=False)['staff_needed']
    .sum()
    .rename(columns={'staff_needed':'total_staff'})
)
total_nxt = (
    df_all_nxt
    .groupby('ds', as_index=False)['staff_needed']
    .sum()
    .rename(columns={'staff_needed':'total_staff'})
)

# 3) Plot
fig3 = px.line(
    total_typ,
    x='ds',
    y='total_staff',
    title="Center‐wide Total Staffing — Typical Week",
    labels={'ds':'Datetime','total_staff':'Total Staff Needed'}
)
fig3.update_xaxes(tickangle=45)
fig3.show()

fig4 = px.line(
    total_nxt,
    x='ds',
    y='total_staff',
    title="Center‐wide Total Staffing — Next Week",
    labels={'ds':'Datetime','total_staff':'Total Staff Needed'}
)
fig4.update_xaxes(tickangle=45)
fig4.show()


In [26]:


# Use to switch thru rooms
room = list(staff_typical.keys())[0]

# build pivot table for Typical Week
df = staff_typical[room].copy()
df['weekday'] = df['ds'].dt.day_name()
df['time']    = df['ds'].dt.strftime('%H:%M')

# pivot: index=time, columns=weekday, values=staff_needed
pivot = df.pivot(index='time', columns='weekday', values='staff_needed')

# ensure Monday→Sunday order
days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
pivot = pivot.reindex(columns=days)

fig = px.imshow(
    pivot,
    labels={'x':'Day of Week','y':'Time of Day','color':'Staff Needed'},
    aspect='auto',
    title=f"Typical Week Staffing Heatmap — {room}"
)
fig.update_xaxes(side='top')
fig.show()


In [27]:
import pandas as pd
import plotly.express as px

# Use to switch thru rooms
room = list(staff_next.keys())[0]

# build pivot table for Next Week
df = staff_next[room].copy()
df['weekday'] = df['ds'].dt.day_name()
df['time']    = df['ds'].dt.strftime('%H:%M')

pivot = df.pivot(index='time', columns='weekday', values='staff_needed')
days  = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
pivot = pivot.reindex(columns=days)

fig = px.imshow(
    pivot,
    labels={'x':'Day of Week','y':'Time of Day','color':'Staff Needed'},
    aspect='auto',
    title=f"Next Week Staffing Heatmap — {room}"
)
fig.update_xaxes(side='top')
fig.show()


In [28]:
import pandas as pd
import plotly.express as px

# 1) Aggregate total staff per timestamp
df_all_typ = pd.concat(
    [v for v in staff_typical.values()],
    ignore_index=True
)
total_typ = (
    df_all_typ
    .groupby('ds', as_index=False)['staff_needed']
    .sum()
    .rename(columns={'staff_needed':'total_staff'})
)

# 2) Extract weekday and time
total_typ['weekday'] = total_typ['ds'].dt.day_name()
total_typ['time']    = total_typ['ds'].dt.strftime('%H:%M')

# 3) Pivot for heatmap
pivot_typ = total_typ.pivot(
    index='time',
    columns='weekday',
    values='total_staff'
)
days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
pivot_typ = pivot_typ.reindex(columns=days)

# 4) Plot
fig = px.imshow(
    pivot_typ,
    labels={'x':'Day of Week','y':'Time of Day','color':'Total Staff'},
    title="Center-wide Typical Week Staffing Heatmap",
    aspect='auto'
)
fig.update_xaxes(side='top')
fig.show()


In [29]:
import pandas as pd
import plotly.express as px

# 1) Aggregate total staff per timestamp
df_all_nxt = pd.concat(
    [v for v in staff_next.values()],
    ignore_index=True
)
total_nxt = (
    df_all_nxt
    .groupby('ds', as_index=False)['staff_needed']
    .sum()
    .rename(columns={'staff_needed':'total_staff'})
)

# 2) Extract weekday and time
total_nxt['weekday'] = total_nxt['ds'].dt.day_name()
total_nxt['time']    = total_nxt['ds'].dt.strftime('%H:%M')

# 3) Pivot for heatmap
pivot_nxt = total_nxt.pivot(
    index='time',
    columns='weekday',
    values='total_staff'
)
days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
pivot_nxt = pivot_nxt.reindex(columns=days)

# 4) Plot
fig = px.imshow(
    pivot_nxt,
    labels={'x':'Day of Week','y':'Time of Day','color':'Total Staff'},
    title="Center-wide Next Week Staffing Heatmap",
    aspect='auto'
)
fig.update_xaxes(side='top')
fig.show()
